In [1]:
cd ..

/Users/ali/Documents/CS tomfoolery/oleorcarl


In [2]:
from src.database.models import Student, db
from src.scraper.items import StudentItem

In [3]:
alistair = Student()
alistair.name = "Alistair Pattison"
alistair.email = "pattisona@carleton.edu"
vars(alistair)

{'__data__': {'name': 'Alistair Pattison', 'email': 'pattisona@carleton.edu'},
 '_dirty': {'email', 'name'},
 '__rel__': {}}

In [4]:
Alistair = StudentItem(alistair)
Alistair["image"] = "12"
display(Alistair)
display(alistair.__data__)

{'name': 'Alistair Pattison', 'email': 'pattisona@carleton.edu', 'image': '12'}

{'name': 'Alistair Pattison', 'email': 'pattisona@carleton.edu', 'image': '12'}

In [13]:
alistair.image = "123"
alistair.__data__

{'name': 'Alistair Pattison',
 'email': 'pattisona@carleton.edu',
 'image': '123'}

In [2]:
from oleorcarl.scraper.spiders.carleton import _get_carleton_cookies

_get_carleton_cookies()

[{'domain': '.carleton.edu',
  'expiry': 1699586315,
  'httpOnly': False,
  'name': '_ga',
  'path': '/',
  'secure': False,
  'value': 'GA1.1.1453914687.1665026315'},
 {'domain': '.carleton.edu',
  'expiry': 1699586315,
  'httpOnly': False,
  'name': '_ga_VNJBV9BSSQ',
  'path': '/',
  'secure': False,
  'value': 'GS1.1.1665026315.1.0.1665026315.0.0.0'},
 {'domain': '.carleton.edu',
  'expiry': 1665026374,
  'httpOnly': False,
  'name': '_gat',
  'path': '/',
  'secure': False,
  'value': '1'},
 {'domain': 'www.carleton.edu',
  'httpOnly': True,
  'name': 'wordpress_sec_2909fed3ec4894e3dcdf70af9ea68ab8',
  'path': '/',
  'secure': True,
  'value': 'pattisona%7C1665199113%7CGeHEdZcChtHlry2aKKbKomcL7dgzVC1jvPCdBqX07DE%7Cecb400f33fe80234c87ee020924814639704ea4128f90764f1c3420cee9a6569'},
 {'domain': '.carleton.edu',
  'expiry': 1665112714,
  'httpOnly': False,
  'name': '_gid',
  'path': '/',
  'secure': False,
  'value': 'GA1.2.475353470.1665026315'},
 {'domain': 'www.carleton.edu',
  'h